# Microarray File Structure

## Imports
- ftplib
  - File Transfer Protocol Library
  - Used for downloading files from FTP servers
- gzip
  - Package used for uncompressing gzipped files
- os
  - Os is a library for working with the file system|

In [ ]:
from ftplib import FTP
import gzip
import os

In [ ]:
d = pd.read_csv(
    'http://pubs.broadinstitute.org/mpr/projects/Leukemia/data_set_ALL_AML_train.txt',
    sep='\t'
    )

cols = d.columns
d = d.reset_index().drop(columns='call.37')
d.columns = cols
d

## Downloading the Data

I found this dataset using NCBI's GEO data portal (https://www.ncbi.nlm.nih.gov/geo/)

Gene expression profile of human colorectal carcinoma (https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE113513)

The URL needs to be split into three parts to work with the FTP package
- HOST_NAME
  - The address for NCBI's ftp server
- HOST_DATA_PATH
  - The path to the studies file on the FTP server
- HOST_FILE_NAME
  - This is the microarray output file

In [ ]:
HOST_NAME = 'ftp.ncbi.nlm.nih.gov'
HOST_DATA_PATH = 'geo/series/GSE113nnn/GSE113513/matrix/'
HOST_FILE_NAME = 'GSE113513_series_matrix.txt.gz'

We can now use the paths we just defined to download the file from the ftp server

Some notes:
- The file is compressed using gzip (.gz) so we will need to use the gzip package to uncompress it to view its contents
- Pandas is able to read .gz files

The downloaded file is saved in the current runtime. When you start a new session you will need to download the file again. You can link your Google Drive to the notebook to save the file there permanently if required.

The lines that start with '!' are part of the header, we will use this to tell pandas to ignore these lines when loading the dataframe.

We will need to load this data separately though, as it specifies the sample types.

In [ ]:
# Same as above
ftp = FTP(HOST_NAME)
ftp.login()
ftp.cwd(HOST_DATA_PATH)

RAW_DATA_DIR = './data/raw/'

# if the ./data/raw/ path does not exist, create it
if not os.path.exists(RAW_DATA_DIR):
    os.makedirs(RAW_DATA_DIR)

# Download the file as a binary file
# RETR <file_name> : means retrieve that file
# open(...) opens a binary file to write the data to
ftp.retrbinary(
    'RETR {}'.format(HOST_FILE_NAME) , 
    open('{}{}'.format(RAW_DATA_DIR, HOST_FILE_NAME), 'wb').write
)

# This looks at the first 75 lines in the file to evaluate the file format

with gzip.open('{}{}'.format(RAW_DATA_DIR, HOST_FILE_NAME)) as fh:
    for i, line in enumerate(fh.readlines()):
        print(line)
        if i == 75:
            break

# Data Processing

## Imports
- gzip
  - We need to read the gzipped file to extract the meta data
- matplotlib
  - We use matplotlib to create some plots and adjust seaborn plots
- numpy
  - We use some of numpys functions for transforming the data
- pandas
  - We use pandas to manipulate the tabular data
- seaborn
  - We use seaborn for statistical plots

  We first import the packages we need. Then we set some global parameters used for generating plots

In [ ]:
# gzip is a pacakge for opening/making gz files
import gzip
# import plt for plotting
import matplotlib.pyplot as plt
# import numpy for Array creation/manipulation
import numpy as np
# import os for working with local files
import os
# pandas is a package for creating / editing data frames
import pandas as pd
# seaborn is a statistical plotting package
import seaborn as sns

# generate plots in the jupyter notebook
%matplotlib inline

# set the size of the figures 
sns.set(rc={'figure.figsize':(12,6), 'figure.dpi': 150})
# set the backgrounds of figures to white with a grid
sns.set_style("whitegrid")

## Reading the Data into a Pandas Dataframe

In the previous section we downloaded the data file from NCBI, we can now load that file into pandas.

Pandas can read compressed files so we do not first need to uncompress it. We will set the comment flag to '!' to ignore those lines in the file. This file is also tab delimited so we will need to set the sep flag to '\t'.

In [ ]:
# Path to folder containing the raw data
RAW_DATA_DIR = './data/raw/'
# File name of the raw data
RAW_DATA_FILENAME = 'GSE113513_series_matrix.txt.gz'

# Load the data into a dataframe
df = pd.read_csv(
    # This creates the full path to the file
    '{}{}'.format(RAW_DATA_DIR, RAW_DATA_FILENAME),
    # Ignore lines that start with '!'
    comment='!',
    # the character used to separate values
    sep='\t',
    
)
# Set the index column of the dataframe to the gene codes
df = df.set_index('ID_REF')
# output the shape of the dataframe
print(df.shape)
# output the first 5 lines of the dataframe
df.head()

## Extracting the Meta data

To extract the metadata we will first load the file into a list. Each element of the list will be one line of the file.



In [ ]:
META_DATA_RAW = []
with gzip.open('{}{}'.format(RAW_DATA_DIR, RAW_DATA_FILENAME)) as fh:
    for line in fh.readlines():
        # convert the line from binary to utf and remove white spaces
        l = line.decode().strip()
        # if the line is not empty and the first character is a '!'
        if l and l[0] == '!':
            META_DATA_RAW.append(l)

# Output the first 5 lines
META_DATA_RAW[:5]

Now that we have a list containing all the meta data we can process it using the function defined below.

This function will output a key / value pair for each line in the metadata. Those key value pairs are then converted into the META_DATA dictionary.

In [ ]:
# Function to split each string into a key and a value
def split_string(string):
    # remove the "!" and split on the first leftmost '\t'
    tmp = string.strip('!').split('\t', 1)
    # if there was a split assign key and value to the two parts
    # else assign the value to None
    key, value = tmp if len(tmp) > 1 else (tmp[0], None)
    # If there is a value
    if value:
        # remove quotation marks and split into a list on the tab
        # this will always create a list
        # if there is no tab the list will have one element
        value = value.replace('"', '').split('\t')
    # return the key and value as a tuple
    return key, value
# map will feed each element of META_DATA into the split_string \
#     function and save the result
# dict converts the map of tuples to a dictionary
#     the first element in the tuple is the key
# .   the second element is the value
META_DATA = dict(map(split_string, META_DATA_RAW))
# output the META_DATA dictionary
print(META_DATA.keys())


This dictionary now contains all the metadata that we will need to label our dataset

In [ ]:
# display the geo accessions using the dict
META_DATA['Sample_geo_accession']

In [ ]:
# display the sample source
META_DATA['Sample_source_name_ch1']

The next step is to combine the sample source with the geo code for each sample so that we can add this information to the pandas dataframe. We can use the zip() function to join the two arrays into a set of key value pairs. The key value pairs can then be used to create a dictionary

In [ ]:
# create a dictionary by merging the geo accessions with the source names
samples = dict(
    # make a tuple containing pairs taken fom the two lists
    # [
    #    (MD['SGA'][0], MD['SSNC'][0]),
    #    (MD['SGA'][1], MD['SSNC'][1]),
    #    ...
    zip(
        META_DATA['Sample_geo_accession'], 
        META_DATA['Sample_source_name_ch1'])
)
# output the dictionary
samples

We now build two lists, one containing the geo accession numbers of normal samples and the other the carcinoma samples.

In [ ]:
# This does a few things
#   1. filter the data to include non-cancerous samples
#        As samples is a dictionary we split it into key-value pairs using .items()
#        .items() -> (key, value)
#        x[1] -> values from the tuple
#   2. the filter function will return a list of tuples
#        for each element in that list return the first item
#        x[0] -> key
norm_samples = list(
    map(lambda x: x[0], 
        filter(
            lambda x: x[1] == 'non-cancerous colorectal tissue', 
            samples.items()
        )
       )
)

# The same as above except for carcinoma samples 
carc_samples = list(
    map(lambda x: x[0], 
        filter(
            lambda x: x[1] == 'colorectal carcinoma tissues', 
            samples.items()
        )
    )
)
# print the lists
print('NORM: ', ','.join(norm_samples))
print()
print('CARC:', ','.join(carc_samples))

### Using the Meta Data on the DataFrame

Now we can use those two lists to subset our dataframe and make sure that the shapes are correct

In [ ]:
# compaare the data frame shapes to make sure it is subsetting correctly
print('full DF', df.shape)
# the normal samples columns
print('norm DF', df[norm_samples].shape)
# the carcinoma sample columns
print('carc DF', df[carc_samples].shape)

Make sure that there are no overlaps between the two classifications.
If there were overlaps this could indicate a mistake in our meta data extraction

In [ ]:
# Make sure that there are no overlaps between sample classes
set(norm_samples).intersection(set(carc_samples))

Check for missing values in the DataFrame

In [ ]:
# Make sure that there are no NA values
print('NA values in DF:', df.isna().sum().sum())

There are no missing values, so we can continue with our analysis

## Data Validation



Create a boxenplot of the data to see the range of the values

This plot does not work well as the outliers are orders of magnitude larger than the mean.

It is interesting that there is that band between ourliers and the distributions.

In [ ]:
sns.boxenplot(data=df, color='#507fbf')
_ = plt.xticks(rotation=90)
_ = plt.xlabel('Sample')
_ = plt.ylabel('Expression')

We would like to log transform the data, but we first should check whether there are any negative values or zeroes.

To do this we can use the agg function to check the min and max values for each sample.


Using a log transform should result in a better representation when using the boxen plot

In [ ]:
df.agg(['min', 'max'])

No that we know that all the values are greater than zero we can perform a log transformation

In [ ]:
# Log transform the data
# This will replace the data in the dataframe with the log value, so we lose the original values
df_log = df.apply(np.log10)

Visualizing the log transformed data.

In [ ]:
# viewing the logged data gives a better picture of the ranges of values
sns.boxenplot(data=df_log, color='#507fbf')
_ = plt.xticks(rotation=90)
_ = plt.xlabel('Sample')
_ = plt.ylabel('Expression (log10)')

For demonstration purposes I will now plot a heatmap of the data.

This will be messy as there is no structure to the way the data is being plotted in terms of expression.

We use the logged data so that the range of values is closer together.

In [ ]:
# Plotting a heatmap off all the expression values for all the samples
_ = sns.heatmap(data=df_log, yticklabels=False)

## Ranking the Genes

Both the ranking method used in the paper, and welch's t-test require the mean and standard deviation for each class across each gene (row).

We can use the agg method to generate a new dataframe containing these metrics for normal and cacinoma samples. These dataframes can then be added onto the original dataframe as new columns.

In [ ]:
# add two new columns containing the mean and std for each row respectively
df_log[['norm_mean', 'norm_std']] = df_log[norm_samples].agg(
    ['mean', 'std'], axis=1
)
# The same as above but for carcinoma samples
df_log[['carc_mean', 'carc_std']] = df_log[carc_samples].agg(
    ['mean', 'std'], axis=1
)
df_log.head()

Now that we have all the required variables we can add a new column containing the result of the t-test.

In [ ]:
import math
# calculate welch's t test using the mean's and std's
#            m_1 - m_2
# t =    __________________
#          ________________
#         |std_1     std_2
#         |-----  +  ----
#       \/ num_1     num2
def welch_t_test(row):
    return (
        (row['norm_mean'] - row['carc_mean']) / 
        np.sqrt(
            row['norm_std']/len(norm_samples) + row['carc_std']/len(carc_samples)
        )
    )
# create a column called similarity containing the results of the t test
df_log['similarity'] = df_log[['norm_mean', 'norm_std', 'carc_mean', 'carc_std']].apply(welch_t_test, axis=1)
df_log.head()

In [ ]:
# sort the data using the similarity
# . once sorted we dont need any of the values used to calculate the similarity
# . This returns a dataframe containing only expression values
df_sorted = df_log.sort_values('similarity').drop(columns=['norm_mean', 'norm_std', 'carc_mean', 'carc_std', 'similarity'])

We can now try plotting the heatmap again using the ranked genes.

Again we do not see much overall, but when we look the the top and bottom rows we can see that there might be some separation.

In [ ]:
# The heatmap sorted by similarity does not show much when looking at all genes
sns.heatmap(data=df_sorted, yticklabels=False)

## Picking the top genes

We can now select the top genes for the two classes using the similarity metric we defined.

In [ ]:
# Create a new dataframe containing the top 25 and bottom 25 genes by similarity
df_features = df_sorted.head(25).append(df_sorted.tail(25))
df_features.shape

Visualizing this subset of the dataframe now shows nice separation when viewed as a heatmap

In [ ]:
# when viewing this heatmap we see that the genes differentiate the data well
sns.heatmap(data=df_features, yticklabels=False, cmap='coolwarm', square=1)

# Classification

## Imports
- matplotlib
- numpy
- pandas
- scikit-learn
  - Logistic Regression
    - The model that we will fit the data to
  - Train Test Split
    - This function will split a dataset into a training and test set
  - plot_confusion_matrix
    - The function that will plot a confusion matrix using the models predictions
  - LeaveOneOut
    - This will implement leave one out cross validation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Logistic regression model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import LeaveOneOut

This redefines the samples classes, these are the same as what we set up earlier using the meta data

In [ ]:
# Dictionary of sample lables calculated earlier
SAMPLES = {
    'GSM3108231': 'non-cancerous colorectal tissue',
    'GSM3108232': 'non-cancerous colorectal tissue',
    'GSM3108233': 'non-cancerous colorectal tissue',
    'GSM3108234': 'non-cancerous colorectal tissue',
    'GSM3108235': 'non-cancerous colorectal tissue',
    'GSM3108236': 'non-cancerous colorectal tissue',
    'GSM3108237': 'non-cancerous colorectal tissue',
    'GSM3108238': 'non-cancerous colorectal tissue',
    'GSM3108239': 'non-cancerous colorectal tissue',
    'GSM3108240': 'non-cancerous colorectal tissue',
    'GSM3108241': 'non-cancerous colorectal tissue',
    'GSM3108242': 'non-cancerous colorectal tissue',
    'GSM3108243': 'non-cancerous colorectal tissue',
    'GSM3108244': 'non-cancerous colorectal tissue',
    'GSM3108245': 'colorectal carcinoma tissues',
    'GSM3108246': 'colorectal carcinoma tissues',
    'GSM3108247': 'colorectal carcinoma tissues',
    'GSM3108248': 'colorectal carcinoma tissues',
    'GSM3108249': 'colorectal carcinoma tissues',
    'GSM3108250': 'colorectal carcinoma tissues',
    'GSM3108251': 'colorectal carcinoma tissues',
    'GSM3108252': 'colorectal carcinoma tissues',
    'GSM3108253': 'colorectal carcinoma tissues',
    'GSM3108254': 'colorectal carcinoma tissues',
    'GSM3108255': 'colorectal carcinoma tissues',
    'GSM3108256': 'colorectal carcinoma tissues',
    'GSM3108257': 'colorectal carcinoma tissues',
    'GSM3108258': 'colorectal carcinoma tissues'
}

In [ ]:
# We will be using the features dataset we defined earlier
# This dataframe contains the 25 top and 25 bottom ranked genes
df_features.head()

## Data Formatting

When using Scikit-Learn, each row should represent a set of observations for a sample.

Currently the samples are represented using the columns, to fix this we can transpose the dataframe.

The new dataframe will have a sample per row, and each column will represent a gene

In [ ]:
# scikit learn needs each row to represent a sample
# for this reason we need to transpose the dataframe
df_features = df_features.transpose()
df_features.head()

Now we should add an extra column that specifies the type of each sample. We will call this column label

In [ ]:
# Add an extra column to the dataframe containing the sample label
df_features['label'] = pd.Series(SAMPLES).astype('category')
df_features.head()

In [ ]:
# Use the category codes to label the samples types using integers
df_features['label_codes'] = df_features["label"].cat.codes
df_features[["label", 'label_codes']]

We will first subset the samples into a train test set to see how the model works.

We will then use leave one out cross validation

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df_features.drop(columns=['label', 'label_codes']), 
    df_features['label_codes'], 
    test_size=0.92, 
    random_state=42)

In [ ]:
# create a new logistic regression model
# There are many different solvers: 
# .  https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
#   They all have their strenghts/weaknesses
lr = LogisticRegression(random_state=0, solver='lbfgs')
# train the model using the training data
clf = lr.fit(X_train, y_train)
# now that the model has been trained, predict the classes in the test set
pred = clf.predict(X_test)
# predict outputs labels, decision_function outputs the confidence scores
y_scores = clf.decision_function(X_test)
# print the accuracy sum(predicted == actual) / len(labels)
print('Predicted Class:\t', pred)
print('Actual Class: \t\t', y_test.to_numpy())

In [ ]:
# y_scores contains the confidence scores
y_scores

In [ ]:
# scoring the model will return the accuracy
# this uses the test features and test labels
clf.score(X_test, y_test)

## Confusion Matrix

A confusion matrix is good method for seeing how well your model made its classifications.

If all the numbers fall along the diagonal it means that all the predictions were correct. If the numbers fall in any of the other blocks it means that they were misclassified

In [ ]:
disp = plot_confusion_matrix(clf, X_test, y_test,
                                 display_labels=df_features['label'].cat.categories,
                                 cmap=plt.cm.Blues,
                                 normalize=None)
disp.ax_.set_title('Confusion Matrix')
disp.ax_.grid(False)
disp.ax_.tick_params(axis='x', rotation=70)

_ = disp.confusion_matrix

## Cross Validation

The LeaveOneOut function will create training and testing sets.

If we have 28 samples, there will be 28 of these sets. Each set will hold out one sample for testing and use the rest to train the model.

When we iterate over the split it will return the indexes for the training and test sets.

We can then use those indexes to subset the data.

In [ ]:
for train_index, test_index in loo.split(df_features):
  print(train_index, test_index)

In [ ]:
# Leave one out cross validation using scikit

# Create a new LeaveOneOut object and assign it to loo
loo = LeaveOneOut()
# Calculate the number of splits, this uses the number of rows in the features set
loo.get_n_splits(df_features)

max_probs = []

# loo.split(features) will return column indexes for train and test sets
# . the train will have len(samples) -1 elements
# . the test set will contain a single index
for train_index, test_index in loo.split(df_features):
    # split the data into train and test sets using the indicies
    df_features, 
    df_features
    X_train = df_features.iloc[train_index].drop(columns=['label', 'label_codes'])
    X_test = df_features.iloc[test_index].drop(columns=['label', 'label_codes'])
    y_train = df_features.iloc[train_index]['label_codes']
    y_test = df_features.iloc[test_index]['label_codes']
    
    # Create a new model each time so that the weights are reset each iteration 
    lr = LogisticRegression(solver='lbfgs')
    # train the model using the training set
    clf = lr.fit(X_train, y_train)
    # print the accuracy of the model using the test set
    print(clf.score(X_test, y_test), end=', ')
    # this will output the probilites per test case
    #   as we only have one test we can use zero to index it
    max_probs.append(max(clf.predict_proba(X_test)[0]))


Using the probabilities we saved in the previous step we can recreate the predictive strength plot from the paper.

In [ ]:
fig = plt.figure(figsize=(2, 4), dpi=150)
g = sns.stripplot(max_probs, orient='v')

# Clustering

## Imports

- numpy
- os
- pandas
- sklearn cluster
  - KMean
  - AgglomerativeClustering

In [ ]:
import numpy as np
import os
import pandas as pd

from sklearn.cluster import KMeans, AgglomerativeClustering

## Loading the Data

We will load the data again in case their were any changes in the previous sections

In [ ]:
# We use the raw data again as clustering should determine the groups
#  If we used the feature set it would bias clusters towards the defined classes

RAW_DATA_DIR = './data/raw/'
RAW_DATA_FILENAME = 'GSE113513_series_matrix.txt.gz'

samples = {
    'GSM3108231': 'non-cancerous colorectal tissue',
    'GSM3108232': 'non-cancerous colorectal tissue',
    'GSM3108233': 'non-cancerous colorectal tissue',
    'GSM3108234': 'non-cancerous colorectal tissue',
    'GSM3108235': 'non-cancerous colorectal tissue',
    'GSM3108236': 'non-cancerous colorectal tissue',
    'GSM3108237': 'non-cancerous colorectal tissue',
    'GSM3108238': 'non-cancerous colorectal tissue',
    'GSM3108239': 'non-cancerous colorectal tissue',
    'GSM3108240': 'non-cancerous colorectal tissue',
    'GSM3108241': 'non-cancerous colorectal tissue',
    'GSM3108242': 'non-cancerous colorectal tissue',
    'GSM3108243': 'non-cancerous colorectal tissue',
    'GSM3108244': 'non-cancerous colorectal tissue',
    'GSM3108245': 'colorectal carcinoma tissues',
    'GSM3108246': 'colorectal carcinoma tissues',
    'GSM3108247': 'colorectal carcinoma tissues',
    'GSM3108248': 'colorectal carcinoma tissues',
    'GSM3108249': 'colorectal carcinoma tissues',
    'GSM3108250': 'colorectal carcinoma tissues',
    'GSM3108251': 'colorectal carcinoma tissues',
    'GSM3108252': 'colorectal carcinoma tissues',
    'GSM3108253': 'colorectal carcinoma tissues',
    'GSM3108254': 'colorectal carcinoma tissues',
    'GSM3108255': 'colorectal carcinoma tissues',
    'GSM3108256': 'colorectal carcinoma tissues',
    'GSM3108257': 'colorectal carcinoma tissues',
    'GSM3108258': 'colorectal carcinoma tissues'
}

In [ ]:
# Read the data into a dataframe
df = pd.read_csv(
    '{}{}'.format(RAW_DATA_DIR, RAW_DATA_FILENAME)
    , comment='!',
    sep='\t',
    
)
df = df.set_index('ID_REF')
df_log = df.apply(np.log)
df_log.head()

## Data Formatting

Once again, we will need to transpose the dataframe so that rows represent samples and columns represent genes.

We will add the labels for each sample as a new column

In [ ]:
# Scikit learn requires samples to be rows in the matrix
#  For this reason we transpose the dataframe
df_log = df_log.transpose()
# We then add the labels to each row
df_log['type'] = pd.Series(samples).astype('category')
df_log['type_codes'] = df_log["type"].cat.codes
df_log.head()

To simplify things we can split the dataframe into two. A features set containing the expression values, and a label set containing the labels for each sample.

In [ ]:
# extract the expression levels as features from the dataframe
features = df_log.drop(columns=['type', 'type_codes']).values
labels = df_log['type_codes'].values
print('features:', features.shape)
print('labels:', labels.shape)

## Clustering Methods

We will use two different clustering methods. KMeans and Agglomerative (hierarchical) clustering. This demonstrates how easy it is to try out different models once your data is formatted correctly.

Both of these methods require you to specify the number of clusters to make. They also have multiple parameters that can be set to fine tune the result.

- https://scikit-learn.org/stable/modules/clustering.html#k-means
- https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering

In [ ]:
# Create a new clustering model and fit the features to it
# K means requires the number of clusters to be specified
kmeans = KMeans(n_clusters=4, random_state=123454321).fit(features)
print('Clustering Labels:\t', kmeans.labels_)
print('Actual Labels:\t\t', labels)

From the results we can see that the cancerous tissue (code 0) all clustered together (cluster 1) while the non-cancerous (code 1) tissues was clustered into three groups (clusters 0, 2, 3)

In [ ]:
# The same process but with a different clustering model
clustering = AgglomerativeClustering(n_clusters=4).fit(features)
print('Clustering Labels:\t', clustering.labels_)
print('Actual Labels:\t\t', labels)

The Agglomerative clustering clustered the cancerous and non-cancerous samples together. But it created two sub-classes for each. code 1 -> (clusters 0, 2) and code 0 -> (custers 1, 3)

To understand the sub clusters we would need more information about source of the samples. Loading in extra data from the Meta Data of the file might allow us to understand why there are the sub clusters. 